# [Projet 3: Développez un moteur de recommandations de films](https://openclassrooms.com/projects/developpez-un-moteur-de-recommandations-de-films)

Link to raw data: [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Moteur+de+recommandation+de+films/imdb-5000-movie-dataset.zip).

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'/Documents/GitHub/OCDataSciencePath/Project3/')    # @home
elif HOST == 'Sirius.local':
    os.chdir(HOME+'Perso/GitHub/OCDataSciencePath/Project3/')         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd

In [2]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToData = HOME+'xxx'    # @home
elif HOST == 'Sirius.local':
    pathToData = HOME+'Downloads/movie_metadata.csv'         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
df = pd.read_csv(pathToData)

In [3]:
n,p = df.shape
print(n,p)
df.sample(5)

5043 28


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
174,Color,Bill Condon,322.0,115.0,386.0,12000.0,Kristen Stewart,21000.0,292298923.0,Adventure|Drama|Fantasy|Romance,...,329.0,English,USA,PG-13,120000000.0,2012.0,17000.0,5.5,2.35,65000
2949,Color,Chris Rock,161.0,102.0,0.0,555.0,Romany Malco,3000.0,25277561.0,Comedy|Romance,...,93.0,English,USA,R,12000000.0,2014.0,966.0,6.5,2.35,0
4327,Color,NaN,19.0,30.0,NaN,424.0,Tim Daly,685.0,NaN,Comedy|Drama,...,56.0,English,USA,NaN,NaN,NaN,511.0,7.3,1.33,1000
807,Color,David Koepp,181.0,107.0,192.0,280.0,Olivia Munn,40000.0,7605668.0,Action|Comedy|Mystery|Romance,...,188.0,English,UK,R,60000000.0,2015.0,2000.0,5.5,2.35,12000
3309,Color,Hong-jin Na,77.0,156.0,43.0,0.0,Jun Kunimura,45.0,770629.0,Fantasy|Horror|Mystery|Thriller,...,24.0,Korean,South Korea,Not Rated,NaN,2016.0,5.0,7.7,2.35,0


In [4]:
for col in df.columns.sort_values():
    miss = 100 * (1 - df[col].count()/n)
    print('{:>30}: {}%'.format(col, miss))

        actor_1_facebook_likes: 0.13880626611144642%
                  actor_1_name: 0.13880626611144642%
        actor_2_facebook_likes: 0.25778306563553066%
                  actor_2_name: 0.25778306563553066%
        actor_3_facebook_likes: 0.4560777315090192%
                  actor_3_name: 0.4560777315090192%
                  aspect_ratio: 6.52389450723776%
                        budget: 9.756097560975608%
     cast_total_facebook_likes: 0.0%
                         color: 0.376759865159626%
                content_rating: 6.008328375966688%
                       country: 0.09914733293674427%
       director_facebook_likes: 2.0622645250842786%
                 director_name: 2.0622645250842786%
                      duration: 0.2974419988102328%
          facenumber_in_poster: 0.25778306563553066%
                        genres: 0.0%
                         gross: 17.52924846321634%
                    imdb_score: 0.0%
                      language: 0.2379535990481907%
     

### Custom database
We start with simple variables:

In [71]:
keepVar = ['movie_title',
           'actor_1_name','actor_2_name','actor_3_name']
wdf = df[keepVar].copy()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
3221,Rosewater,Numan Acar,Claire Foy,Haluk Bilginer
2013,Waterloo,Rod Steiger,Dan O'Herlihy,Jack Hawkins
3645,Men of War,Catherine Bell,Kevin Tighe,Tim Guinee
4103,DysFunktional Family,Eddie Griffin,Joe Howard,Robert Noble
20,The Hobbit: The Battle of the Five Armies,Aidan Turner,Adam Brown,James Nesbitt


Needed post-processing:
- lowerise all text to avoid missing doublons
- doublons/approximative doublons in `movie_title`,`actor_N_name`
- try one-hot encode the three `actor_N_name` (ignore order)


Notes:
- film titles (at least) have `\xa0` unbreakable spaces in the end: don't forget it when lookinf for titles in code!

In [72]:
# lowerise

toLowerVar = ('movie_title','actor_1_name','actor_2_name','actor_3_name')
for col in toLowerVar:
    wdf[col] = wdf[col].str.lower()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
2596,madison,bruce dern,jake lloyd,william shockley
745,the happening,zooey deschanel,alan ruck,kristen connolly
4158,young frankenstein,madeline kahn,peter boyle,teri garr
1787,ghosts of mars,jason statham,clea duvall,natasha henstridge
4345,the marine 4: moving target,paul mcgillion,mike 'the miz' mizanin,david lewis


In [73]:
# remove pure duplicates

print(wdf.duplicated().sum())
# wdf.loc[wdf.duplicated()].sort_values(by='movie_title')
wdf.drop_duplicates(inplace=True)

123


In [74]:
# look specifically at the rows where only film title is duplicated

print(wdf['movie_title'].duplicated(keep=False).sum())
wdf.loc[wdf['movie_title'].duplicated(keep=False)].sort_values(by='movie_title')

6


,movie_title,actor_1_name,actor_2_name,actor_3_name
1852,brothers,natalie portman,jake gyllenhaal,bailee madison
2882,brothers,natalie portman,jake gyllenhaal,ethan suplee
3791,out of the blue,raymond burr,don gordon,jim byrnes
3840,out of the blue,william kircher,matthew sunderland,paul glover
1002,the host,j.d. evermore,chandler canterbury,rachel roberts
2988,the host,doona bae,kang-ho song,ah-sung ko


In [75]:
# no obvious error: don't want to check which one is correct... just drop them

wdf.drop_duplicates(subset='movie_title',inplace=True)
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
4158,young frankenstein,madeline kahn,peter boyle,teri garr
4745,drinking buddies,olivia wilde,anna kendrick,ti west
3177,teacher's pet,nathan lane,kelsey grammer,jerry stiller
2867,dirty work,don rickles,jack warden,traylor howard
2287,deconstructing harry,woody allen,lynn cohen,judy davis


In [76]:
# one hot encode the names (actually it is a poor idea for testing but let's continue...)

toOneHotVar = ('actor_1_name','actor_2_name','actor_3_name')
allActor = []
for col in toOneHotVar:
    allActor += list(wdf[col])
allActor = np.unique(allActor)

In [77]:
# beware: some films do not have all actors filled. Remove them.

wdf = wdf.dropna(subset=toOneHotVar,how='any')
wdf.shape

(4894, 4)

In [78]:
for i,a in enumerate(allActor): # speed this up!
    value = np.full((wdf.shape[0],),False)
    for col in toOneHotVar:
        value = value | (wdf[col] == a)    
    wdf[i] = 1*value

In [79]:
# wdf.drop(list(toOneHotVar),axis=1,inplace=True)
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name,0,1,2,3,4,5,...,6241,6242,6243,6244,6245,6246,6247,6248,6249,6250
0,avatar,cch pounder,joel david moore,wes studi,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4125,the eclipse,aidan quinn,iben hjejle,jim norton,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2140,the transporter,jason statham,qi shu,matt schulze,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2389,fear and loathing in las vegas,johnny depp,michael jeter,ellen barkin,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4109,freeway,bokeem woodbine,dan hedaya,robert peters,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# check all films have 3 actors

(wdf.sum(axis=1) != 3).sum()

0

Dump the table

In [82]:
path,nameext= os.path.split(pathToData)
name,ext = os.path.splitext(nameext)

wdf.to_csv(os.path.join(path,name+'_CLEAN'+ext),index=False)